# Vector similarity search using Neon Postgres

This notebook guides you through using [Neon Serverless Postgres](https://neon.tech/) as a vector database for OpenAI embeddings. It demonstrates how to:

1. Use embeddings created by OpenAI API.
2. Store embeddings in a Neon Serverless Postgres database.
3. Convert a raw text query to an embedding with OpenAI API.
4. Use Neon with the `pgvector` extension to perform vector similarity search.

## SECRETS !!!

### Prepare your OpenAI API key

An OpenAI API key is required to generate vectors for documents and queries.

If you do not have an OpenAI API key, obtain one from https://platform.openai.com/account/api-keys.

Add the OpenAI API key as an operating system environment variable or provide it for the session when prompted. If you define an environment variable, name the variable `OPENAI_API_KEY`.

For information about configuring your OpenAI API key as an environment variable, refer to [Best Practices for API Key Safety](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety).

In [ ]:
import os, dotenv
dotenv.load_dotenv()

## Retrieve sample embeddings data

This guide uses pre-computed Wikipedia article embeddings available in the OpenAI Cookbook `examples` directory so that you do not have to compute embeddings with your own OpenAI credits. 

Import the pre-computed embeddings zip file:
Extract the downloaded zip file:

In [ ]:
import zipfile
import os
import wget
import re
import tempfile

current_directory = os.getcwd()

# Check to see if the csv file was extracted
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)

if os.path.exists(file_path):
    print(f"The csv file {file_name} already exists in the data directory.")

else:
    print(f"Downloading Zip")
    # The file is ~700 MB. Importing it will take several minutes.
    embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"
    wget.download(embeddings_url)
    
    print(f"Unzipping  {file_name}.")
    zip_file_path = os.path.join(current_directory,"../../data", "vector_database_wikipedia_articles_embedded.zip")
    output_directory = os.path.join(current_directory, "../../data")

    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(output_directory)
    if os.path.exists(file_path):
        print(f"The csv file {file_name} DOES exist in the data directory.")
    else:
        print(f"The csv file {file_name} does NOT exist in the data directory. CANNOT CONTINUE.")


## Prerequisites

Before you begin, ensure that you have the following:

1. A Neon Postgres database. You can create an account and set up a project with a ready-to-use `neondb` database in a few simple steps. For instructions, see [Sign up](https://neon.tech/docs/get-started-with-neon/signing-up) and [Create your first project](https://neon.tech/docs/get-started-with-neon/setting-up-a-project).
2. A connection string for your Neon database. You can copy it from the **Connection Details** widget on the Neon **Dashboard**. See [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).
3. The `pgvector` extension. Install the extension in Neon by running `CREATE EXTENSION vector;`. For instructions, see [Enable the pgvector extension](https://neon.tech/docs/extensions/pgvector#enable-the-pgvector-extension). 
4. Your [OpenAI API key](https://platform.openai.com/account/api-keys).
5. Python and `pip`.

### Install required modules

This notebook requires the `openai`, `psycopg2`, `pandas`, `wget`, and `python-dotenv` packages. You can install them with `pip`:


In [ ]:
! pip install openai==0.28.1 psycopg2 pandas wget python-dotenv

### Test your OpenAPI key

In [ ]:
# Test to ensure that your OpenAI API key is defined as an environment variable or provide it when prompted
# If you run this notebook locally, you may have to reload the terminal and the notebook to make the environment available

import os
from getpass import getpass

# Check if OPENAI_API_KEY is set as an environment variable
if os.getenv("OPENAI_API_KEY") is not None:
    print("Your OPENAI_API_KEY is ready")
else:
    # If not, prompt for it
    api_key = getpass("Enter your OPENAI_API_KEY: ")
    if api_key:
        print("Your OPENAI_API_KEY is now available for this session")
        # Optionally, you can set it as an environment variable for the current session
        os.environ["OPENAI_API_KEY"] = api_key
    else:
        print("You did not enter your OPENAI_API_KEY")

## Connect to your Neon database

Provide your Neon database connection string below or define it in an `.env` file using a `DATABASE_URL` variable. For information about obtaining a Neon connection string, see [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).

In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# The connection string can be provided directly here.
# Replace the next line with Your Neon connection string.
connection_string = None
database_password = os.environ.get("DATABASE_PASSWORD")

# If connection_string is not directly provided above, 
# then check if DATABASE_URL is set in the environment or .env.
if not connection_string:
    connection_string = os.environ.get("DATABASE_URL")

    # If neither method provides a connection string, raise an error.
    if not connection_string:
        raise ValueError("Please provide a valid connection string either in the code or in the .env file as DATABASE_URL.")

# Connect using the connection string
connection = psycopg2.connect(connection_string,password=database_password)

# Create a new cursor object
cursor = connection.cursor()

# Execute this query to test the database connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Your database connection was successful!")
else:
    print("Your connection failed.")

## Create a table and add indexes for your vector embeddings

The vector table created in your database is called **articles**. Each object has **title** and **content** vectors. 

An index is defined on both the **title** and **content** vector columns.

In [ ]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS public.articles (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INTEGER
);

ALTER TABLE public.articles ADD PRIMARY KEY (id);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.articles USING ivfflat (content_vector) WITH (lists = 1000);

CREATE INDEX ON public.articles USING ivfflat (title_vector) WITH (lists = 1000);
'''
if False:
    # Execute the SQL statements
    cursor.execute(create_table_sql)
    cursor.execute(create_indexes_sql)

    # Commit the changes
    connection.commit()

## Upload the data

Load the pre-computed vector data into your `articles` table from the `.csv` file. There are 25000 records, so expect the operation to take several minutes.

In [ ]:
import io

# Check the size of the data
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
if result != None and result[0] == 0:

    # Create the COPY command for copy_expert
    copy_command = '''COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id) FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');'''

    # Define a generator function to process the csv file
    def process_file(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                yield line

    # Path to your local CSV file
    csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'



    print(f"Preparing to read CSV file...")
    # Create a StringIO object to store the modified lines
    modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))
    print(f"CSV file has been loaded to memory.")


    print(f"Uploading CSV formatted data from memory...")
    # Execute the COPY command using copy_expert
    cursor.copy_expert(copy_command, modified_lines) #type: ignore
    print(f"CSV formatted data has been uploaded from memory.")


    print(f"Prepare to commit the transaction...")
    # Commit the changes
    connection.commit()
    print(f"Transaction committed.")

Check the number of records to ensure the data has been been loaded. There should be 25000 records.

In [ ]:
# Check the size of the data
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
if result != None: print(f"Record count on database:{result[0]}")

## Search your data

After the data is stored in your Neon database, you can query the data for nearest neighbors. 

Start by defining the `query_neon` function, which is executed when you run the vector similarity search. The function creates an embedding based on the user's query, prepares the SQL query, and runs the SQL query with the embedding. The pre-computed embeddings that you loaded into your database were created with `text-embedding-ada-002` OpenAI model, so you must use the same model to create an embedding for the similarity search.

A `vector_name` parameter is provided that allows you to search based on "title" or "content".

In [ ]:
import openai

def query_neonDB(query, collection_name, vector_name="title_vector", top_k=10, operador="-"):

     # Create an embedding vector from the user query
    x = openai.Embedding.create(input=query, model="text-embedding-ada-002")
    embedded_query = x["data"][0]["embedding"] #type: ignore
    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create the SQL query
    query_sql = f"""
    SELECT id, url, title, l2_distance({vector_name},'{embedded_query_pg}'::VECTOR(1536)) AS similarity
    FROM {collection_name}
    ORDER BY {vector_name} {operador} '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

Run a similarity search based on both `title_vector` and `content_vector` embeddings:

In [ ]:
import openai
query_text = "gatitos angora"
query_operator = "<#>"
load_dotenv()

# Query based on `title_vector` embeddings
query_results = query_neonDB(query=query_text, collection_name="Articles", operador=query_operator)
print(f"Results for {query_text} from TITLE ONLY using {query_operator}:")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

print("")

# Query based on `content_vector` embeddings
print(f"Results for {query_text} from TITLE AND CONTENTS using {query_operator}:")
query_results = query_neonDB(query=query_text, collection_name="Articles", vector_name="content_vector", operador=query_operator)
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})   -  {result[1]}  ")

Vector query reference: https://neon.tech/docs/extensions/pgvector#enable-the-pgvector-extension